## IPL Data Description and Ingestion

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

## Loading the dataset

Data is provided in *csv* format. Spark provides *sqlContext.read.csv* to read the csv data and create the dataframe. 
Spark can automatically infer the schema from the data if *inferSchema* is set to True.

In [0]:
%fs ls /FileStore/tables/ipl_datasets/

path,name,size,modificationTime
dbfs:/FileStore/tables/ipl_datasets/deliveries.csv,deliveries.csv,15442270,1678101160000
dbfs:/FileStore/tables/ipl_datasets/matches.csv,matches.csv,117096,1678101115000


### Read records using custom schema

In [0]:
from pyspark.sql.types import *

# Define schema for matches.csv
matches_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("season", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("date", StringType(), True),
    StructField("team1", StringType(), True),
    StructField("team2", StringType(), True),
    StructField("toss_winner", StringType(), True),
    StructField("toss_decision", StringType(), True),
    StructField("result", StringType(), True),
    StructField("dl_applied", IntegerType(), True),
    StructField("winner", StringType(), True),
    StructField("win_by_runs", IntegerType(), True),
    StructField("win_by_wickets", IntegerType(), True),
    StructField("player_of_match", StringType(), True),
    StructField("venue", StringType(), True),
    StructField("umpire1", StringType(), True),
    StructField("umpire2", StringType(), True),
    StructField("umpire3", StringType(), True)
])


In [0]:
matches_df = sqlContext.read.csv("dbfs:/FileStore/tables/ipl_datasets/matches.csv", 
                                header = True, 
                                schema = StructType(matches_schema) )
matches_df.cache()
matches_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- season: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- result: string (nullable = true)
 |-- dl_applied: integer (nullable = true)
 |-- winner: string (nullable = true)
 |-- win_by_runs: integer (nullable = true)
 |-- win_by_wickets: integer (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)
 |-- umpire3: string (nullable = true)



In [0]:
matches_df.show(5)

+---+------+---------+----------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+--------------+-------------+-------+
| id|season|     city|      date|               team1|               team2|         toss_winner|toss_decision|result|dl_applied|              winner|win_by_runs|win_by_wickets|player_of_match|               venue|       umpire1|      umpire2|umpire3|
+---+------+---------+----------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+--------------+-------------+-------+
|  1|  2017|Hyderabad|2017-04-05| Sunrisers Hyderabad|Royal Challengers...|Royal Challengers...|        field|normal|         0| Sunrisers Hyderabad|         35|             0|   Yuvraj Singh|Rajiv Gandhi Inte...|   AY Dandekar|     NJ Llong|   nu

#### Memory Consumed to store the dataframe

In [0]:
spark._jvm.org.apache.spark.util.SizeEstimator.estimate(matches_df._jdf)

Out[10]: 149376080

#### Partitions the dataframe has

The dataframes are internally stores as RDDs and partitions. To find number of partitions.

In [0]:
matches_df.rdd.getNumPartitions()

Out[11]: 1

In [0]:
# Define schema for deliveries.csv
deliveries_schema = StructType([
    StructField("match_id", IntegerType(), True),
    StructField("inning", IntegerType(), True),
    StructField("batting_team", StringType(), True),
    StructField("bowling_team", StringType(), True),
    StructField("over", IntegerType(), True),
    StructField("ball", IntegerType(), True),
    StructField("batsman", StringType(), True),
    StructField("non_striker", StringType(), True),
    StructField("bowler", StringType(), True),
    StructField("is_super_over", IntegerType(), True),
    StructField("wide_runs", IntegerType(), True),
    StructField("bye_runs", IntegerType(), True),
    StructField("legbye_runs", IntegerType(), True),
    StructField("noball_runs", IntegerType(), True),
    StructField("penalty_runs", IntegerType(), True),
    StructField("batsman_runs", IntegerType(), True),
    StructField("extra_runs", IntegerType(), True),
    StructField("total_runs", IntegerType(), True),
    StructField("player_dismissed", StringType(), True),
    StructField("dismissal_kind", StringType(), True),
    StructField("fielder", StringType(), True)
])

In [0]:
deliveries_df = sqlContext.read.csv("dbfs:/FileStore/tables/ipl_datasets/deliveries.csv", 
                                header = True, 
                                schema = StructType(deliveries_schema) )
deliveries_df.cache()
deliveries_df.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- inning: integer (nullable = true)
 |-- batting_team: string (nullable = true)
 |-- bowling_team: string (nullable = true)
 |-- over: integer (nullable = true)
 |-- ball: integer (nullable = true)
 |-- batsman: string (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- is_super_over: integer (nullable = true)
 |-- wide_runs: integer (nullable = true)
 |-- bye_runs: integer (nullable = true)
 |-- legbye_runs: integer (nullable = true)
 |-- noball_runs: integer (nullable = true)
 |-- penalty_runs: integer (nullable = true)
 |-- batsman_runs: integer (nullable = true)
 |-- extra_runs: integer (nullable = true)
 |-- total_runs: integer (nullable = true)
 |-- player_dismissed: string (nullable = true)
 |-- dismissal_kind: string (nullable = true)
 |-- fielder: string (nullable = true)



#### Memory Consumed to store the dataframe

In [0]:
spark._jvm.org.apache.spark.util.SizeEstimator.estimate(deliveries_df._jdf)

Out[12]: 149926624

#### Partitions the dataframe has

The dataframes are internally stores as RDDs and partitions. To find number of partitions.

In [0]:
deliveries_df.rdd.getNumPartitions()

Out[13]: 4